In [1]:
# MSc Project: Biometric Security - Baseline Model
# Author: Stella Williams
# Date: 04.10.2025

# ----------------------------------------
# Step 1: Load Pretrained FaceNet Model
# ----------------------------------------
import torch
from facenet_pytorch import InceptionResnetV1
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
import random, os, io, cv2
from PIL import Image
from tqdm import tqdm

#use gpu if possible

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

#load pretained facenet model

model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

Using device: cpu


In [2]:
'''# Step 2: Load & Filter LFW Dataset (e.g. 5 identities)
# Path to dataset
lfw_path = '/Users/stel/Documents/Dissertation/msc-biometric-security/Datasets/lfw-dataset'

# Define transforms
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

# Load full dataset
lfw_full = datasets.ImageFolder(root=lfw_path, transform=transform)

# Pick 5 identities only (simulate SME-scale)
target_classes = random.sample(lfw_full.classes, 5)
print("Selected classes:", target_classes)

# Filter indices
target_indices = [i for i, (_, label) in enumerate(lfw_full) if lfw_full.classes[label] in target_classes]
lfw_subset = Subset(lfw_full, target_indices)
# Define the loader
lfw_loader = DataLoader(lfw_subset, batch_size=1, shuffle=True)
'''

# ----------------------------------------
# Step 2: Load & Filter LFW Dataset (5 identities)
# ----------------------------------------
lfw_path = '/Users/stel/Documents/Dissertation/msc-biometric-security/Datasets/lfw-dataset'
# Define transforms
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])
# Load full dataset
lfw_full = datasets.ImageFolder(root=lfw_path, transform=transform)
target_classes = random.sample(lfw_full.classes, 5)
print("Selected classes:", target_classes)
# Filter indices
target_idx = [i for i, (_, lab) in enumerate(lfw_full) if lfw_full.classes[lab] in target_classes]
lfw_subset = Subset(lfw_full, target_idx)
# Define the loader
lfw_loader = DataLoader(lfw_subset, batch_size=1, shuffle=True)


Selected classes: ['Jose_Woldenberg', 'William_Hyde', 'Jose_Serra', 'Joseph_Hoy', 'Cecilia_Bolocco']


In [3]:
'''
# Step 3: FGSM Attack

import torch.nn as nn

# Loss and epsilon
criterion = nn.CosineEmbeddingLoss()
epsilon = 0.05

def fgsm_attack(image, gradient, eps):
    # Sign of gradient
    sign = gradient.sign()
    return torch.clamp(image + eps * sign, 0, 1)

# Get a sample
image, label = next(iter(lfw_loader))
image, label = image.to(device), label.to(device)

# Enable gradient
image.requires_grad = True

# Get embedding of true image
embedding_orig = model(image)

# Create "true" target for cosine loss (match itself)
target = torch.tensor([1.0]).to(device)

# Compute loss
loss = criterion(embedding_orig, embedding_orig.detach(), target)
model.zero_grad()
loss.backward()

# FGSM
perturbed_image = fgsm_attack(image, image.grad.data, epsilon)

# Evaluate perturbed embedding
embedding_adv = model(perturbed_image)

# Cosine similarity
cos = nn.CosineSimilarity(dim=1)
sim = cos(embedding_orig, embedding_adv).item()
print(f"Cosine similarity between clean and adversarial: {sim:.4f}")'''


# ----------------------------------------
# Step 3: FGSM Attack
# ----------------------------------------
import torch.nn as nn
# Loss and epsilon
criterion = nn.CosineEmbeddingLoss()
cos = nn.CosineSimilarity(dim=1)

def fgsm_attack(image, grad, eps):
    return torch.clamp(image + eps * grad.sign(), 0, 1)

# Sample one image for display
image, label = next(iter(lfw_loader))
image, label = image.to(device), label.to(device)
# Enable gradient
image.requires_grad = True

# Get embedding of true image
embedding_orig = model(image)
# Create "true" target for cosine loss (match itself)
target = torch.tensor([1.0]).to(device)
# Compute loss
loss = criterion(embedding_orig, embedding_orig.detach(), target)
model.zero_grad()
loss.backward()
# FGSM
perturbed_image = fgsm_attack(image, image.grad.data, eps=0.05)
# Evaluate perturbed embedding
embedding_adv = model(perturbed_image)
# Cosine similarity
print(f"FGSM cosine similarity: {cos(embedding_orig, embedding_adv).item():.4f}")




Cosine similarity between clean and adversarial: 1.0000


In [2]:
'''# Step 4: Defence via JPEG Compression (Fixed)
import cv2
from PIL import Image

def jpeg_defence(img_tensor, quality=50):
    # Detach from computation graph
    img_np = img_tensor.detach().squeeze().permute(1, 2, 0).cpu().numpy()
    img_np = (img_np * 255).astype(np.uint8)

    # Save to JPEG in memory
    _, encoded = cv2.imencode('.jpg', img_np, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    decoded = cv2.imdecode(encoded, cv2.IMREAD_COLOR)

    # Convert back to tensor
    recovered = transforms.ToTensor()(Image.fromarray(cv2.cvtColor(decoded, cv2.COLOR_BGR2RGB)))
    return recovered.unsqueeze(0).to(device)

# Apply defence
jpeg_image = jpeg_defence(perturbed_image)

# Evaluate similarity again
embedding_jpeg = model(jpeg_image)
sim_defended = cos(embedding_orig, embedding_jpeg).item()
print(f"Cosine similarity (clean vs. JPEG-recovered): {sim_defended:.4f}") '''

# ----------------------------------------
# Step 4: JPEG Defence
# ----------------------------------------
def jpeg_defence(img_tensor, quality=50):
    # Detach from computation graph
    img_np = img_tensor.detach().squeeze().permute(1, 2, 0).cpu().numpy()
    # Save to JPEG in memory
    img_np = (img_np * 255).astype(np.uint8)
    _, enc = cv2.imencode('.jpg', img_np, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    dec = cv2.imdecode(enc, cv2.IMREAD_COLOR)
    # Convert back to tensor
    rec = transforms.ToTensor()(Image.fromarray(cv2.cvtColor(dec, cv2.COLOR_BGR2RGB)))
    return rec.unsqueeze(0).to(device)
# Apply defence
jpeg_image = jpeg_defence(perturbed_image)
# Evaluate similarity again
embedding_jpeg = model(jpeg_image)
print(f"JPEG defence cosine similarity: {cos(embedding_orig, embedding_jpeg).item():.4f}")




NameError: name 'perturbed_image' is not defined

In [ ]:
# ----------------------------------------
# Step 5: Visualise Images
# ----------------------------------------
def show_images(images, titles):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img.detach().squeeze().permute(1, 2, 0).cpu())
        ax.set_title(title)
        ax.axis('off')
    plt.tight_layout(); plt.show()

show_images([image, perturbed_image, jpeg_image],
            ['Original', 'FGSM Adversarial', 'JPEG Defence'])


In [1]:
'''# Step 6: Benchmark Loop – FGSM + JPEG Defence Evaluation
from tqdm import tqdm

# Parameters
epsilon = 0.05
num_samples = 30
sim_threshold = 0.7  # if cosine similarity drops below this, identity is 'lost'

# Results
attack_success = 0
jpeg_recovery = 0
skipped = 0
similarities = []

model.eval()

for i in tqdm(range(num_samples), desc="Benchmarking FGSM + JPEG"):

    # Get sample image
    image, label = next(iter(lfw_loader))
    image, label = image.to(device), label.to(device)
    image.requires_grad = True

    # Embedding of clean image
    embed_orig = model(image)

    # Skip if model already fails
    pred_clean = embed_orig.detach().cpu().numpy()
    if np.linalg.norm(pred_clean) == 0:
        skipped += 1
        continue

    # Compute gradient
    target = torch.tensor([1.0]).to(device)
    loss = criterion(embed_orig, embed_orig.detach(), target)
    model.zero_grad()
    loss.backward()

    # FGSM attack
    perturbed = fgsm_attack(image, image.grad.data, epsilon)
    embed_adv = model(perturbed)
    
    # Cosine similarity drop = attack success
    sim_orig_adv = cos(embed_orig, embed_adv).item()
    if sim_orig_adv < sim_threshold:
        attack_success += 1

    # Defence: JPEG
    jpeg = jpeg_defence(perturbed)
    embed_jpeg = model(jpeg)
    sim_orig_jpeg = cos(embed_orig, embed_jpeg).item()

    if sim_orig_jpeg >= sim_threshold:
        jpeg_recovery += 1

    similarities.append((sim_orig_adv, sim_orig_jpeg))

# Results
print("\n📊 Benchmark Summary")
print(f"Samples evaluated: {num_samples}")
print(f"Skipped samples (no match): {skipped}")
print(f"FGSM attack success rate: {attack_success / (num_samples - skipped):.2f}")
print(f"JPEG defence recovery rate: {jpeg_recovery / (num_samples - skipped):.2f}")
'''

# ----------------------------------------
# Step 6: FGSM Benchmark Loop
# ----------------------------------------
# Parameters
epsilons = [0.05, 0.1, 0.15, 0.3]
print("\n### FGSM Benchmark ###")

for eps in epsilons:
    attack_success = 0; recovery_success = 0; total = 0
    
    for i in range(30):
        img, lbl = next(iter(lfw_loader))
        img, lbl = img.to(device), lbl.to(device)
        img.requires_grad = True
        
        embed_orig = model(img)
        target = torch.tensor([1.0]).to(device)
        loss = criterion(embed_orig, embed_orig.detach(), target)
        model.zero_grad(); loss.backward()
        
        adv = fgsm_attack(img, img.grad.data, eps)
        embed_adv = model(adv)
        sim_adv = cos(embed_orig, embed_adv).item()
        
        if sim_adv < 0.8: attack_success += 1
        
        jpeg = jpeg_defence(adv)
        sim_jpeg = cos(embed_orig, model(jpeg)).item()
        if sim_jpeg > 0.9: recovery_success += 1
        
        total += 1
    
    print(f"ε={eps:.2f} | Attack Success: {attack_success/total:.2f} | JPEG Recovery: {recovery_success/total:.2f}")


NameError: name 'model' is not defined

In [ ]:
# ----------------------------------------
# ✅ Step 7: PGD Attack Function
# ----------------------------------------
def pgd_attack(model, image, epsilon=0.1, alpha=0.01, num_iter=10):
    adv = image.clone().detach().to(device)
    adv.requires_grad = True
    
    for _ in range(num_iter):
        embed = model(adv)
        target = torch.tensor([1.0]).to(device)
        loss = criterion(embed, model(image).detach(), target)
        model.zero_grad(); loss.backward()
        grad = adv.grad.data
        
        adv = adv + alpha * grad.sign()
        perturb = torch.clamp(adv - image, -epsilon, epsilon)
        adv = torch.clamp(image + perturb, 0, 1).detach_()
        adv.requires_grad = True
        
    return adv

In [3]:
# ----------------------------------------
# ✅ Step 8: PGD Test Run
# ----------------------------------------
print("\n### PGD Test ###")

img, lbl = next(iter(lfw_loader))
img, lbl = img.to(device), lbl.to(device)

pert_pgd = pgd_attack(model, img, epsilon=0.1, alpha=0.01, num_iter=20)
sim_pgd = cos(model(img), model(pert_pgd)).item()
print(f"PGD cosine similarity: {sim_pgd:.4f}")

jpeg_pgd = jpeg_defence(pert_pgd)
sim_pgd_jpeg = cos(model(img), model(jpeg_pgd)).item()
print(f"PGD + JPEG defence similarity: {sim_pgd_jpeg:.4f}")

show_images([img, pert_pgd, jpeg_pgd],
            ['Original', 'PGD Attack', 'JPEG Defence (PGD)'])


### PGD Test ###


NameError: name 'lfw_loader' is not defined

In [1]:
'''epsilons = [0.05, 0.1, 0.15, 0.3]

for eps in epsilons:
    print(f"\n📌 Testing FGSM with ε = {eps}")
    attack_success = 0
    recovery_success = 0
    total = 0

    for i in range(30):  # 30 test samples
        image, label = next(iter(lfw_loader))
        image, label = image.to(device), label.to(device)
        image.requires_grad = True

        # Original embedding and prediction
        embed_orig = model(image)
        pred_orig = embed_orig.argmax(dim=1) if embed_orig.dim() > 1 else None

        # Generate adversarial example
        target = torch.tensor([1.0]).to(device)
        loss = criterion(embed_orig, embed_orig.detach(), target)
        model.zero_grad()
        loss.backward()
        adv_image = fgsm_attack(image, image.grad.data, eps)

        # Evaluate adversarial
        embed_adv = model(adv_image)
        sim = cos(embed_orig, embed_adv).item()
        if sim < 0.8:  # below threshold = attack success
            attack_success += 1

        # Apply JPEG
        jpeg_image = jpeg_defence(adv_image)
        embed_jpeg = model(jpeg_image)
        sim_jpeg = cos(embed_orig, embed_jpeg).item()
        if sim_jpeg > 0.9:  # above threshold = successful recovery
            recovery_success += 1

        total += 1

    print(f"FGSM ε={eps} → Attack Success Rate: {attack_success/total:.2f}")
    print(f"FGSM ε={eps} → JPEG Recovery Rate: {recovery_success/total:.2f}")
'''


📌 Testing FGSM with ε = 0.05


NameError: name 'lfw_loader' is not defined